In [1]:
import os
from moviepy.editor import (
    VideoFileClip,
    clips_array,
    TextClip,
    CompositeVideoClip,
    AudioFileClip,
    ImageClip,
    concatenate_videoclips,
)
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import pandas as pd

os.environ[
    "IMAGEMAGICK_BINARY"
] = r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"
from moviepy.config import change_settings

# Specify the path to the ImageMagick binary (replace with your path)
change_settings(
    {"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"}
)

In [2]:
phrases = pd.read_csv("./Phrases.csv")

In [3]:
# Define the input frames directory, audio file, and caption text
frames_directory = "Frames\Frame2"  # Replace with the path to your frames
audio_file = "./Audio.mp3"  # Replace with the path to your audio file
captions = list(phrases["Phrase"])  # Replace with your captions

In [4]:
refresh_directions = [
    "moving left-left",
    "moving left",
    "moving up-left",
    "moving up",
    "moving forward",
    "moving up-right",
    "moving right",
    "moving right-right",
    "moving down-right",
    "moving down",
    "moving backward",
    "moving down-left",
]

directions = {
    "moving left": 1,
    "moving right": 2,
    "moving up": 3,
    "moving down": 4,
    "moving forward": 5,
    "moving backward": 6,
    "moving up-left": 7,
    "moving up-right": 8,
    "moving down-left": 9,
    "moving down-right": 10,
    "moving left-left": 11,
    "moving right-right": 12,
}

In [5]:
len_words = [len(sentence.split()) for sentence in phrases["Phrase"]]
second = 2
frame_persistance_time = [(len_word / second) for len_word in len_words]
fps = 24
num_frames = [
    int(frame_persistance_time[i] * fps) for i in range(len(frame_persistance_time))
]
num_frames_per_direction = [
    num_frame / len(refresh_directions) for num_frame in num_frames
]

In [6]:
image_files = []

In [7]:
k = 0
for i in range(phrases.shape[0]):
    for j in range(int(num_frames_per_direction[k])):
        a = (-1) ** j
        refresh_directions = refresh_directions[::a]
        for direction in refresh_directions:
            image_files.append(
                f"Frames\\Frame2\\{i+1}_{j+1}_{directions[direction]}.png"
            )
    k += 1

In [8]:
len(image_files)

7212

In [9]:
# Sort the image files based on their filenames
# image_files.sort()

# Load the audio file
audio = AudioFileClip(audio_file)

# Initialize an empty list to store caption clips
caption_clips = []

In [10]:
# Create caption clips
for caption in captions:
    caption_clip = TextClip(caption, fontsize=20, color="white", bg_color="black")
    caption_clip = caption_clip.set_position(("left", "bottom"))
    caption_clips.append(caption_clip)

# Calculate the duration for each caption based on the total duration and the number of captions
duration_per_caption = audio.duration / len(captions)

# Create a sequence of caption clips with the correct timing
sequence_clips = []
start_time = 0

for caption_clip in caption_clips:
    caption_clip = caption_clip.set_duration(duration_per_caption).set_start(start_time)
    sequence_clips.append(caption_clip)
    start_time += duration_per_caption

# Create a CompositeVideoClip from the sequence of caption clips
captions_composite = CompositeVideoClip(sequence_clips, size=(512, 512))

# Set the audio of the captions_composite to match the audio_clip
captions_composite = captions_composite.set_audio(audio)

In [11]:
# Create a list of image frames as video frames
video_frames = [
    ImageClip(img, duration=audio.duration / len(image_files)) for img in image_files
]

In [14]:
vf_concatinated = concatenate_videoclips(video_frames, method="compose")

final_video = CompositeVideoClip([vf_concatinated, captions_composite])

final_video = final_video.set_duration(audio.duration)

final_video = final_video.set_audio(audio)

fps = 3

final_video.write_videofile("GenesisAI_prototype_lv2_uni.mp4", codec="libx264", fps=fps)

Moviepy - Building video GenesisAI_prototype_lv2_uni.mp4.
MoviePy - Writing audio in GenesisAI_prototype_lv2_uniTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video GenesisAI_prototype_lv2_uni.mp4



Moviepy - Done !
Moviepy - video ready GenesisAI_prototype_lv2_uni.mp4
